<a href="https://colab.research.google.com/github/xuandy05/Python-A.I./blob/master/Cart_Pole_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install pyglet==1.3.2 > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display as ipythondisplay
from IPython.display import clear_output

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()


xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
#Create and Train the Model

In [0]:
from keras.models import Sequential
model = Sequential()

from keras.layers import Dense
model.add(Dense(24, input_dim = 4, activation = 'relu'))
model.add(Dense(24, activation = 'relu'))
model.add(Dense(2, activation = 'linear'))

from keras.optimizers import adam
model.compile(optimizer=adam(lr = 0.001), loss='mse')

Using TensorFlow backend.
W0808 23:10:52.208764 140538343139200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 23:10:52.250842 140538343139200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 23:10:52.262151 140538343139200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 23:10:52.319598 140538343139200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
from collections import deque
import random
env = gym.make("CartPole-v1")
gamma = 0.95
epsilon = 1
memory = deque(maxlen = 2000)
epochs = 1000
for i in range(epochs):
  state = env.reset()
  state = np.reshape(state, [1,4])
  done = False
  for k in range(1500):
    e = random.uniform(0,1)
    if e < epsilon:
      action = env.action_space.sample()
    else:
      pred = model.predict(state)
      action = np.argmax(pred)
    next_state, reward, done, info = env.step(action)
    next_state = np.reshape(next_state, [1, 4])
    memory.append((state, action, reward, next_state, done))
    state = next_state 
    if done:
      print("Epoch:", i, "Score:", k)
      break
  epsilon = min(0.1, epsilon*0.995) 
  batchsize = 32
  if len(memory) > batchsize:
    batch = random.sample(memory, batchsize)
    for state, action, reward, next_state, done in batch:
      target = reward
      if not done:
        target = reward + gamma*(max(model.predict(next_state)[0]))
      target_f = model.predict(state)
      target_f[0][action] = target
      model.fit(state, target_f, epochs = 1, verbose = 0)
      
env.close()

Epoch: 0 Score: 13
Epoch: 1 Score: 169
Epoch: 2 Score: 198
Epoch: 3 Score: 168
Epoch: 4 Score: 157
Epoch: 5 Score: 188
Epoch: 6 Score: 102
Epoch: 7 Score: 328
Epoch: 8 Score: 333
Epoch: 9 Score: 146
Epoch: 10 Score: 289
Epoch: 11 Score: 163
Epoch: 12 Score: 241
Epoch: 13 Score: 215
Epoch: 14 Score: 180
Epoch: 15 Score: 147
Epoch: 16 Score: 114
Epoch: 17 Score: 164
Epoch: 18 Score: 447
Epoch: 19 Score: 193
Epoch: 20 Score: 241
Epoch: 21 Score: 217
Epoch: 22 Score: 139
Epoch: 23 Score: 391
Epoch: 24 Score: 173
Epoch: 25 Score: 245
Epoch: 26 Score: 28
Epoch: 27 Score: 201
Epoch: 28 Score: 195
Epoch: 29 Score: 166
Epoch: 30 Score: 245
Epoch: 31 Score: 230
Epoch: 32 Score: 200
Epoch: 33 Score: 18
Epoch: 34 Score: 242
Epoch: 35 Score: 187
Epoch: 36 Score: 209
Epoch: 37 Score: 451
Epoch: 38 Score: 234
Epoch: 39 Score: 499
Epoch: 40 Score: 252
Epoch: 41 Score: 221
Epoch: 42 Score: 258
Epoch: 43 Score: 175
Epoch: 44 Score: 245
Epoch: 45 Score: 238
Epoch: 46 Score: 204
Epoch: 47 Score: 329
Epoch

In [0]:
#Display
env = wrap_env(gym.make('CartPole-v1'))
state = env.reset()
for time_t in range(500):
    env.render()
    state = np.reshape(state, [1,4])
    pred2 = model.predict(state)
    action = np.argmax(pred2)
    state, reward, done, _ = env.step(action)
    if done:
        break
        
print(time_t)
env.close()
show_video()


NameError: ignored